# SVD та LU Декомпозиція: Продакшн Приклади

## Наукові джерела

### SVD (Singular Value Decomposition):
1. **Zhang, Z. (2015).** "The Singular Value Decomposition, Applications and Beyond" - arXiv:1510.08532
   - Застосування в machine learning, matrix completion, і data classification

2. **Sarwar et al. (2000, 2002).** Рекомендаційні системи на основі SVD для MovieLens dataset
   - Використання SVD для collaborative filtering

3. **Deerwester et al. (1990), Berry et al. (1995).** Latent Semantic Analysis (LSA)
   - SVD для аналізу текстів та пошуку документів

4. **Henry et al. (2002).** "Application of SVD to Macromolecular X-Ray Data" - PMC1302779
   - Застосування в кристалографії та біології

### LU Декомпозиція:
1. **Bartels & Golub (1969).** "The simplex method using LU decomposition" - Communications of the ACM
   - Застосування в linear programming

2. **Toloo et al. (2015).** "LU Decomposition in DEA" - Computational Economics
   - Використання для Data Envelopment Analysis в аналізі ефективності лікарень

3. **Stabrowski (2018).** "Parallel LU decomposition: Gauss vs. Crout algorithm"
   - Реальні застосування з Florida University Matrix Collection

---

In [3]:
# Імпорт необхідних бібліотек
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from PIL import Image
import requests
from io import BytesIO

print("✅ Бібліотеки завантажено")

✅ Бібліотеки завантажено


# Частина 2: LU Декомпозиція - Фінансове Моделювання (Production)

## Приклад: Оптимізація портфелю інвестицій

**Джерело:** Bartels & Golub (1969), "Simplex method using LU decomposition"

**Продакшн використання:**
- Фінансові інститути (Goldman Sachs, BlackRock)
- Розв'язування тисяч систем рівнянь щодня
- Перерахунок портфелів при зміні ринкових умов

In [4]:
# Задача: Розподіл $1,000,000 між різними активами
# Обмеження:
#   - Мінімум 20% у облігаціях (низький ризик)
#   - Максимум 40% у акціях (високий ризик)
#   - Решта в нерухомість
#   - Цільова прибутковість: 8%

# Система рівнянь:
# x + y + z = 1,000,000  (весь капітал)
# 0.05x + 0.12y + 0.08z = 80,000  (8% прибутковість)
# x = 200,000  (20% в облігаціях)

A = np.array([
    [1.0, 1.0, 1.0],      # Загальна сума
    [0.05, 0.12, 0.08],   # Прибутковість
    [1.0, 0.0, 0.0]       # Облігації = 200k
])

b = np.array([1_000_000, 80_000, 200_000])

print("Система рівнянь (Ax = b):")
print("A:")
print(A)
print(f"\nb: {b}")

Система рівнянь (Ax = b):
A:
[[1.   1.   1.  ]
 [0.05 0.12 0.08]
 [1.   0.   0.  ]]

b: [1000000   80000  200000]


In [7]:
# LU Декомпозиція
P, L, U = linalg.lu(A)

print("\nLU Декомпозиція:")
print("\nP (Permutation matrix):")
print(P)
print("\nL (Lower triangular):")
print(L)
print("\nU (Upper triangular):")
print(U)
print("\n✅ Перевірка: A = PLU")
print(np.allclose(A, P @ L @ U))


LU Декомпозиція:

P (Permutation matrix):
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]

L (Lower triangular):
[[ 1.    0.    0.  ]
 [ 1.    1.    0.  ]
 [ 0.05 -0.07  1.  ]]

U (Upper triangular):
[[ 1.    1.    1.  ]
 [ 0.   -1.   -1.  ]
 [ 0.    0.   -0.04]]

✅ Перевірка: A = PLU
True


In [8]:
# Розв'язуємо систему в два кроки
# Крок 1: Forward substitution (Ly = Pb)
Pb = P @ b
y = linalg.solve_triangular(L, Pb, lower=True)
print("Крок 1 - Forward substitution (Ly = Pb):")
print(f"y = {y}")

# Крок 2: Backward substitution (Ux = y)
x = linalg.solve_triangular(U, y, lower=False)
print("\nКрок 2 - Backward substitution (Ux = y):")
print(f"x = {x}")

print("\n💰 Розподіл інвестицій:")
print(f"  Облігації (5% прибутковість):  ${x[0]:,.0f} ({x[0]/1_000_000*100:.1f}%)")
print(f"  Акції (12% прибутковість):     ${x[1]:,.0f} ({x[1]/1_000_000*100:.1f}%)")
print(f"  Нерухомість (8% прибутковість): ${x[2]:,.0f} ({x[2]/1_000_000*100:.1f}%)")
print(f"\n  Очікуваний дохід: ${(0.05*x[0] + 0.12*x[1] + 0.08*x[2]):,.0f}")

Крок 1 - Forward substitution (Ly = Pb):
y = [1000000. -800000.  -26000.]

Крок 2 - Backward substitution (Ux = y):
x = [200000. 150000. 650000.]

💰 Розподіл інвестицій:
  Облігації (5% прибутковість):  $200,000 (20.0%)
  Акції (12% прибутковість):     $150,000 (15.0%)
  Нерухомість (8% прибутковість): $650,000 (65.0%)

  Очікуваний дохід: $80,000


In [9]:
# PRODUCTION SCENARIO: Зміна ринкових умов
# Припустимо, прибутковість акцій змінилася з 12% на 10%
# LU вже обчислено, просто змінюємо b!

print("\n🔄 PRODUCTION BONUS: Швидкий перерахунок при зміні умов\n")

import time

# Нові цільові прибутковості
new_targets = [70_000, 75_000, 80_000, 85_000, 90_000]

print("Метод 1: Повне розв'язування (без LU):")
start = time.time()
for target in new_targets:
    b_new = np.array([1_000_000, target, 200_000])
    x_new = np.linalg.solve(A, b_new)
time_without_lu = time.time() - start
print(f"  Час: {time_without_lu*1000:.3f} ms")

print("\nМетод 2: Використання існуючого LU:")
start = time.time()
for target in new_targets:
    b_new = np.array([1_000_000, target, 200_000])
    Pb = P @ b_new
    y = linalg.solve_triangular(L, Pb, lower=True)
    x_new = linalg.solve_triangular(U, y, lower=False)
time_with_lu = time.time() - start
print(f"  Час: {time_with_lu*1000:.3f} ms")

print(f"\n✅ Прискорення: {time_without_lu/time_with_lu:.2f}x")
print(f"\n  У продакшні (1000 перерахунків на день):")
print(f"    Без LU: {time_without_lu*1000*1000:.0f} ms = {time_without_lu*1000:.2f} секунд")
print(f"    З LU:   {time_with_lu*1000*1000:.0f} ms = {time_with_lu*1000:.2f} секунд")
print(f"    Економія часу: {(time_without_lu-time_with_lu)*1000:.2f} секунд/день")


🔄 PRODUCTION BONUS: Швидкий перерахунок при зміні умов

Метод 1: Повне розв'язування (без LU):
  Час: 0.856 ms

Метод 2: Використання існуючого LU:
  Час: 1.425 ms

✅ Прискорення: 0.60x

  У продакшні (1000 перерахунків на день):
    Без LU: 856 ms = 0.86 секунд
    З LU:   1425 ms = 1.42 секунд
    Економія часу: -0.57 секунд/день


# Порівняння SVD vs LU

| Критерій | SVD | LU |
|----------|-----|----|
| **Застосування** | Рекомендації, стиснення, ML | Розв'язування систем, оптимізація |
| **Складність** | O(mn²) | O(n³) |
| **Коли використовувати** | Аналіз даних, зменшення розмірності | Багаторазові розв'язування |
| **Продакшн приклади** | Netflix, Spotify, Amazon | Фінанси, інженерія, моделювання |
| **Головна перевага** | Знаходить приховані патерни | Швидке повторне розв'язування |

## Висновки

**SVD:**
- ✅ Ідеально для великих розріджених матриць (sparse matrices)
- ✅ Знаходить латентні фактори в даних
- ✅ Дозволяє компресію з мінімальною втратою точності
- 📊 Використовується у: рекомендаційних системах, обробці зображень, NLP

**LU:**
- ✅ Дуже ефективно для багаторазових розв'язувань
- ✅ Числово стабільніше за прямі методи
- ✅ Легко паралелізується
- 📊 Використовується у: фінансовому моделюванні, інженерних симуляціях, оптимізації

## Додаткові ресурси

1. **SVD Tutorial:** https://arxiv.org/abs/1510.08532
2. **Netflix Prize:** https://www.netflixprize.com/
3. **LAPACK Documentation:** http://www.netlib.org/lapack/
4. **Numerical Recipes:** Chapter 2.3 (LU) and 2.6 (SVD)